In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16314574
paper_name = 'sliwa_korona_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Sheet1', header=None)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 12 x 3


In [10]:
original_data.head()

,0,1,2
0,YIL001W,2,1.012
1,YIL006W,2,1.005
2,YIL041W,3,1.013
3,YIL077C,5,1.010
4,YIL087C,6,1.011


In [11]:
original_data['orf'] = original_data[0].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [0, 1, 2, orf]
Index: []


In [15]:
original_data['data'] = original_data[2] - 1

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(12, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [16201]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,16201
data_type,value
orf,
YGR035C,0.011
YHL014C,0.010
YIL001W,0.012
YIL006W,0.005
YIL041W,0.013


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16201
,data_type,value
gene_id,orf,
2980,YGR035C,0.011
931,YHL014C,0.010
1188,YIL001W,0.012
1193,YIL006W,0.005
1228,YIL041W,0.013


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id       16201           
data_type        value     valuez
gene_id orf                      
2980    YGR035C  0.011  20.258107
931     YHL014C  0.010  18.416461
1188    YIL001W  0.012  22.099754
1193    YIL006W  0.005   9.208231
1228    YIL041W  0.013  23.941400

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 16314574...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00, 10.33it/s]

Updating the data_modified_on field...
